In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
import time

In [3]:
s = time.time()
a = pd.read_csv('train/train_app.csv')
b = pd.read_csv('train/train_sms.csv')
c = pd.read_csv('train/train_user.csv')
d = pd.read_csv('train/train_voc.csv')

a1 = pd.read_csv('test/test_app.csv')
b1 = pd.read_csv('test/test_sms.csv')
c1 = pd.read_csv('test/test_user.csv')
d1 = pd.read_csv('test/test_voc.csv')

print('耗时',(time.time() -s))
# IMEI 手机序列号

耗时 29.91897988319397


In [9]:
c.shape

(6106, 13)

In [23]:
# 改变名字
b.calltype_id = b.calltype_id.astype(np.object)
b.rename(columns={'calltype_id':'sms_calltype_id'},inplace=True)
b1.calltype_id = b1.calltype_id.astype(np.object)
b1.rename(columns={'calltype_id':'sms_calltype_id'},inplace=True)

d.calltype_id = d.calltype_id.astype(np.object)
d.rename(columns={'calltype_id':'voice_calltype_id'},inplace=True)
d1.calltype_id = d1.calltype_id.astype(np.object)
d1.rename(columns={'calltype_id':'voice_calltype_id'},inplace=True)

In [39]:
for dtype in ['float', 'int', 'object']:
    # 选中对应的dtype列
    selected_dtype = a.select_dtypes(include=[dtype])
    # 查看内存使用量的平均值
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    # 获取到的数据单位为K, 这里转换一下
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns:{:03.2f} MB".format(dtype, mean_usage_mb))

Average memory usage for float columns:0.00 MB
Average memory usage for int columns:0.00 MB
Average memory usage for object columns:120.47 MB


,phone_no_m,busi_name,flow,month_id
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,NaN,0.000000,NaN
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,NaN,0.000000,NaN
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,NaN,0.021529,2019-12
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,微信,0.001846,2019-12
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,高德导航,0.002288,2019-12


In [47]:
d.select_dtypes(include=['object','category']).describe()

,phone_no_m,opposite_no_m,voice_calltype_id,start_datetime,city_name,county_name,imei_m
count,5015430,5015430,5015430,5015430,4455901,4455901,5015430
unique,6025,1259878,3,4079930,21,196,14570
top,21e490b5531874a711456555dd3f7d9dd58a2a9d233087...,5825f76d4fb2e58e2ad70af2230ff63d714c00e444bfc0...,1,2020-03-04 14:19:33,成都,高新南区,a6664bcfdc892ab75e55e4a46c63cf727c838ce34664db...
freq,18261,9709,3008933,7,2018589,374546,18261


In [48]:
numberics = ['int16', 'int32','int64', 'float16', 'float32', 'float64']
objects = ['month_id','sms_calltype_id','phone_no_m','city_name','county_name','voice_calltype_id','county_name','busi_name','imei_m']
def memory_reduce(df):
#     import pdb; pdb.set_trace()
    start_memory = df.memory_usage(deep=True).sum()/1024**2
    for col in df.columns:
        # 转换object 为category
        if col in objects:
            df[col] = df[col].astype('category')
            continue
        
        col_type = df[col].dtypes
        if col_type in numberics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
            elif str(col_type)[:5] == 'float':
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.float64).max:
                        df[col] = df[col].astype(np.float64)
    end_memory = df.memory_usage(deep=True).sum()/1024**2
    print('Mem. usage decreased to {:5.2f} Mb {:.1f}% reduction'.format(end_memory, 100*(start_memory - end_memory)/start_memory))
    return df



In [49]:
a = memory_reduce(a)
b = memory_reduce(b)
c = memory_reduce(c)
d = memory_reduce(d)

a1 = memory_reduce(a1)
b1 = memory_reduce(b1)
c1 = memory_reduce(c1)
d1 = memory_reduce(d1)

Mem. usage decreased to 29.96 Mb 88.7% reduction
Mem. usage decreased to 1725.48 Mb 0.0% reduction
Mem. usage decreased to  1.39 Mb 0.0% reduction
Mem. usage decreased to 1300.66 Mb 40.1% reduction
Mem. usage decreased to  1.57 Mb 85.7% reduction
Mem. usage decreased to 103.38 Mb 0.0% reduction
Mem. usage decreased to  0.48 Mb 0.0% reduction
Mem. usage decreased to 72.46 Mb 39.7% reduction


In [50]:
a.to_pickle('after_reducion/train_app.pkl')
b.to_pickle('after_reducion/train_sms.pkl')
c.to_pickle('after_reducion/train_user.pkl')
d.to_pickle('after_reducion/train_voc.pkl')

a1.to_pickle('after_reducion/test_app.pkl')
b1.to_pickle('after_reducion/test_sms.pkl')
c1.to_pickle('after_reducion/test_user.pkl')
d1.to_pickle('after_reducion/test_voc.pkl')



In [51]:
del a,b,c,d,a1,b1,c1,d1
gc.collect()

3598